In [1]:
import numpy as np 
import pandas as pd

df = pd.read_csv('customers.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [11]:
value_counts = df['y'].value_counts()
value_counts

y
no     39922
yes     5289
Name: count, dtype: int64

In [13]:
from imblearn.over_sampling import RandomOverSampler

input_columns = df.drop('y', axis=1)

class_column = df['y']

oversampler = RandomOverSampler(random_state=0)

input_columns_resampled, class_column_resampled = oversampler.fit_resample(input_columns, class_column)

df = pd.concat([input_columns_resampled, class_column_resampled], axis=1)

In [15]:
class_distribution = df['y'].value_counts()
class_distribution

y
no     39922
yes    39922
Name: count, dtype: int64

In [17]:
X = df.iloc[:,0:16].values

y = df.iloc[:,-1]

In [21]:
from sklearn.preprocessing import LabelEncoder

labelencoder_y = LabelEncoder()

y = labelencoder_y.fit_transform(y)
print(y)

[0 0 0 ... 1 1 1]


In [23]:
X_job = X[:,[1]]
print(np.unique(X_job))

['admin.' 'blue-collar' 'entrepreneur' 'housemaid' 'management' 'retired'
 'self-employed' 'services' 'student' 'technician' 'unemployed' 'unknown']


In [27]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])],
                       sparse_threshold=0)

X_job = ct.fit_transform(X_job)
print(X_job.shape)
print(X_job)

(79844, 12)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


In [29]:
X_cat = X[:,[1, 2, 3, 4, 6, 7, 8, 10, 15]]

In [31]:
X_cat.shape

(79844, 9)

In [35]:
orginalNumOfCols = X_cat.shape[1]

for i in range(X_cat.shape[1]): 
    
    currNumOfCols = X_cat.shape[1]
    
    indexOfColumnToEncode = currNumOfCols - orginalNumOfCols + i
    
    ct = ColumnTransformer(transformers=
                            [('encoder', 
                              OneHotEncoder(), [indexOfColumnToEncode])], 
                           remainder='passthrough', 
                           sparse_threshold=0)
    
    X_cat = ct.fit_transform(X_cat)

In [37]:
X_cat.shape

(79844, 44)

In [39]:
X_num = X[:,[0, 5, 9, 11, 12, 13, 14]]

X = np.concatenate((X_num,X_cat), axis=1)

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [43]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [45]:
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential()

classifier.add(Dense(units = 128, activation = 'relu'))

classifier.add(Dense(units = 64, activation = 'relu'))

classifier.add(Dense(units = 32, activation = 'relu'))

classifier.add(Dense(units = 1, activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifierHistory = classifier.fit(X_train, y_train, epochs = 100, validation_data=(X_test, y_test))

Epoch 1/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 3s 911us/step - accuracy: 0.8292 - loss: 0.3832 - val_accuracy: 0.8683 - val_loss: 0.3120
Epoch 2/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 2s 830us/step - accuracy: 0.8795 - loss: 0.2936 - val_accuracy: 0.8811 - val_loss: 0.2932
Epoch 3/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 2s 813us/step - accuracy: 0.8886 - loss: 0.2766 - val_accuracy: 0.8843 - val_loss: 0.2815
Epoch 4/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 2s 796us/step - accuracy: 0.8997 - loss: 0.2519 - val_accuracy: 0.8947 - val_loss: 0.2659
Epoch 5/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 2s 918us/step - accuracy: 0.9075 - loss: 0.2349 - val_accuracy: 0.8978 - val_loss: 0.2625
Epoch 6/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 2s 884us/step - accuracy: 0.9160 - loss: 0.2188 - val_accuracy: 0.9035 - val_loss: 0.2550
Epoch 7/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 1s 787us/step - accuracy: 0.9196 - loss: 0.2081 - val_accuracy: 0.9092 - val_loss: 0.2401
Epoch 8/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 1s 784us/step - accuracy: 

In [47]:
from keras.layers import Dropout

classifier = Sequential()

classifier.add(Dense(units = 128, activation = 'relu'))

classifier.add(Dense(units = 64, activation = 'relu'))

classifier.add(Dropout(0.5))

classifier.add(Dense(units = 32, activation = 'relu'))

classifier.add(Dense(units = 1, activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifierHistory = classifier.fit(X_train, y_train, epochs = 100, validation_data=(X_test, y_test)) 

Epoch 1/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7999 - loss: 0.4361 - val_accuracy: 0.8656 - val_loss: 0.3234
Epoch 2/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 2s 905us/step - accuracy: 0.8698 - loss: 0.3147 - val_accuracy: 0.8769 - val_loss: 0.3013
Epoch 3/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 2s 848us/step - accuracy: 0.8793 - loss: 0.2960 - val_accuracy: 0.8814 - val_loss: 0.2960
Epoch 4/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 2s 817us/step - accuracy: 0.8847 - loss: 0.2856 - val_accuracy: 0.8850 - val_loss: 0.2879
Epoch 5/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 2s 848us/step - accuracy: 0.8875 - loss: 0.2784 - val_accuracy: 0.8873 - val_loss: 0.2804
Epoch 6/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 2s 837us/step - accuracy: 0.8923 - loss: 0.2676 - val_accuracy: 0.8872 - val_loss: 0.2812
Epoch 7/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 2s 856us/step - accuracy: 0.8972 - loss: 0.2606 - val_accuracy: 0.8934 - val_loss: 0.2690
Epoch 8/100
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 2s 824us/step - accuracy: 0.